In [1]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=bcc0e805ac7988192f401a55cba94fb6d01040bd8c8a5fe946ab0a1536b61641
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
 

In [3]:
import pandas as pd
from googletrans import Translator
import re
import time

# Function to detect Hindi text
def is_hindi(text):
    if pd.isna(text):
        return False
    hindi_range = re.compile(r'[\u0900-\u097F]+')
    return bool(hindi_range.search(str(text)))

# Read the CSV file
df = pd.read_csv('/content/IndianFoodDatasetCSV.csv')

# Initialize translator
translator = Translator()

# Prepare a list of texts to translate, only including Hindi text
texts_to_translate = []
indices_to_translate = []
for index, text in enumerate(df['Instructions']):
    if is_hindi(text):
        texts_to_translate.append(text)
        indices_to_translate.append(index)

# Translate in batches
batch_size = 100
for i in range(0, len(texts_to_translate), batch_size):
    batch = texts_to_translate[i:i+batch_size]
    try:
        translations = translator.translate(batch, src='hi', dest='en')
        for j, translation in enumerate(translations):
            texts_to_translate[i+j] = translation.text
        print(f"Translated batch {i//batch_size + 1}/{len(texts_to_translate)//batch_size + 1}")
        time.sleep(1)  # Short delay between batches
    except Exception as e:
        print(f"Error in batch {i//batch_size + 1}: {e}")

# Update the dataframe with translations
df['TranslatedInstructions'] = df['Instructions']  # Create a new column with original instructions
for index, translation in zip(indices_to_translate, texts_to_translate):
    df.at[index, 'TranslatedInstructions'] = translation

# Save the updated CSV file
df.to_csv('indian_food_recipes_english.csv', index=False)

print("Translation complete. New file saved as 'indian_food_recipes_english.csv'")

Translated batch 1/13
Translated batch 2/13
Translated batch 3/13
Translated batch 4/13
Translated batch 5/13
Translated batch 6/13
Translated batch 7/13
Translated batch 8/13
Translated batch 9/13
Translated batch 10/13
Translated batch 11/13
Translated batch 12/13
Translated batch 13/13
Translation complete. New file saved as 'indian_food_recipes_english.csv'


In [8]:
# Remove specified columns
columns_to_remove = ['RecipeName', 'Ingredients', 'Instructions']
# df = df.drop(columns=columns_to_remove)

# Display the first few rows of the updated DataFrame
df.head()

# Display the list of remaining columns
print("\nRemaining columns:")
print(df.columns.tolist())

# Save the updated DataFrame to a new CSV file
df.to_csv('indian_food_recipes_updated.csv', index=False)
print("\nUpdated DataFrame saved to 'indian_food_recipes_updated.csv'")


Remaining columns:
['Srno', 'TranslatedRecipeName', 'TranslatedIngredients', 'PrepTimeInMins', 'CookTimeInMins', 'TotalTimeInMins', 'Servings', 'Cuisine', 'Course', 'Diet', 'TranslatedInstructions', 'URL']

Updated DataFrame saved to 'indian_food_recipes_updated.csv'


In [9]:
# Load the CSV file
udf = pd.read_csv('indian_food_recipes_updated.csv')

# Initialize translator
translator = Translator()

# Prepare a list of texts to translate
texts_to_translate = []
cells_to_update = []

for column in udf.columns:
    for index, cell in enumerate(udf[column]):
        if is_hindi(cell):
            texts_to_translate.append(str(cell))
            cells_to_update.append((index, column))

# Translate in batches
batch_size = 100
for i in range(0, len(texts_to_translate), batch_size):
    batch = texts_to_translate[i:i+batch_size]
    try:
        translations = translator.translate(batch, src='hi', dest='en')
        for j, translation in enumerate(translations):
            index, column = cells_to_update[i+j]
            udf.at[index, column] = translation.text
        print(f"Translated batch {i//batch_size + 1}/{len(texts_to_translate)//batch_size + 1}")
        time.sleep(1)  # Short delay between batches
    except Exception as e:
        print(f"Error in batch {i//batch_size + 1}: {e}")

# Save the updated CSV file
udf.to_csv('indian_food_recipes_fully_english.csv', index=False)

print("Translation complete. New file saved as 'indian_food_recipes_fully_english.csv'")
print(f"Total cells translated: {len(texts_to_translate)}")

# Display a sample of updated rows
print("\nSample of updated rows:")
print(udf.sample(5))

Translated batch 1/6
Translated batch 2/6
Translated batch 3/6
Translated batch 4/6
Translated batch 5/6
Translated batch 6/6
Translation complete. New file saved as 'indian_food_recipes_fully_english.csv'
Total cells translated: 569

Sample of updated rows:
       Srno                               TranslatedRecipeName  \
5436   8019                     Pakistani Roasted Gosht Recipe   
6210  10182  Coorg Style Kaad Manga Curry Recipe -Sweet & S...   
4404   5907                      Cheese Masala Omelette Recipe   
3862   5013    Bengali Doi Maach Recipe (Fish In Yogurt Curry)   
3196   3933            Lauki Moongphali Soup (Recipe In Hindi)   

                                  TranslatedIngredients  PrepTimeInMins  \
5436  750 grams Mutton - cut into medium pieces,6 On...              10   
6210  4 Mango (Ripe) - small variety,5 Green Chillie...              15   
4404  2 Whole Eggs,2 tablespoons Onion - finely chop...              10   
3862  400 grams Rohu fish - sliced,1 Onion -